In [9]:
from entity.key_management_entity import KeyManagementEntity

In [10]:
kme_1 = KeyManagementEntity(
        kme_id="kme-1",
        account_id="3001",
        ca_cert="certs/account-3001-server-ca-qukaydee-com.crt",
        client_cert=("certs/sae-1.crt", "certs/sae-1.key"),
    )

kme_2 = KeyManagementEntity(
        kme_id="kme-2",
        account_id="3001",
        ca_cert="certs/account-3001-server-ca-qukaydee-com.crt",
        client_cert=("certs/sae-2.crt", "certs/sae-2.key"),
    )

In [11]:
print("--- Getting key from sae-1 ---")
status_code, keys = kme_1.get_key("sae-2", 1, 1024)
print(f"Status: {status_code}")

key_data = keys["keys"][0]
key_id_from_kme1 = key_data["key_ID"]
key_from_kme_1 = key_data["key"]
print(f"Key ID: {key_id_from_kme1}")
print(f"Key: {key_from_kme_1}")

--- Getting key from sae-1 ---
Status: 200
Key ID: 572c1c3d-a5c1-46fa-8404-a120b88ef921
Key: lpaQzB4tQy6aaO/oVHe2BJHg1TiJwjbmKHZylkGtHSOBgIsRlDI7ykX9NlN5Wbm1jfYnREj5n8Zj7oWXA37CoDHCoNSlDFLifOaKT1DjYWvcDGEEA3A1Jy6YxQxconxbq69ikI4yGSBfaGwlPSLXoRtC+4EIjyB0jWCOwbeOTio=


In [12]:
# sae-2 (kme_2) retrieves key shared with sae-1
print("\n--- Retrieving key from sae-2 ---")
status_code, keys = kme_2.get_key_with_id("sae-1", key_id_from_kme1)
print(f"Status: {status_code}")

key_data = keys["keys"][0]
key_from_kme_2 = key_data["key"]
print(f"Key: {key_from_kme_2}")


--- Retrieving key from sae-2 ---
Status: 200
Key: lpaQzB4tQy6aaO/oVHe2BJHg1TiJwjbmKHZylkGtHSOBgIsRlDI7ykX9NlN5Wbm1jfYnREj5n8Zj7oWXA37CoDHCoNSlDFLifOaKT1DjYWvcDGEEA3A1Jy6YxQxconxbq69ikI4yGSBfaGwlPSLXoRtC+4EIjyB0jWCOwbeOTio=


In [13]:
print("\n--- KME1から取得した鍵とKME2から取得した鍵が一致するか確認 ---")
assert key_from_kme_1 == key_from_kme_2
print("OK")


--- KME1から取得した鍵とKME2から取得した鍵が一致するか確認 ---
OK
